# ห้องปฏิบัติการเสริม - เครือข่ายประสาทเทียมแบบง่าย

ในห้องปฏิบัติการนี้ เราจะสร้างเครือข่ายประสาทเทียมขนาดเล็กโดยใช้ TensorFlow
   <center> <img  src="./images/C2_W1_CoffeeRoasting_v1.png" width="400" />   <center/>


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_common.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_neurons_utils.py'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_coffee_utils.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('lab_neurons_utils.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_coffee_utils.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from lab_utils_common import dlc
from lab_coffee_utils import load_coffee_data, plt_roast, plt_prob, plt_layer, plt_network, plt_output_unit
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


## DataSet (ชุดข้อมูล)

In [ ]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

มาวาดกราฟข้อมูลการคั่วกาแฟที่อยู่ด้านล่างกันเถอะ คุณสมบัติทั้งสองคือ อุณหภูมิเป็นองศาเซลเซียสและระยะเวลาเป็นนาที [Coffee Roasting at Home](https://www.merchantsofgreencoffee.com/how-to-roast-green-coffee-in-your-oven/) แนะนำว่าระยะเวลาที่ดีที่สุดควรอยู่ระหว่าง 12 ถึง 15 นาที ในขณะที่อุณหภูมิควรอยู่ระหว่าง 175 ถึง 260 องศาเซลเซียส แน่นอนว่า เมื่ออุณหภูมิสูงขึ้น ระยะเวลาก็ควรลดลง

In [ ]:
plt_roast(X,Y)

### ทำให้ข้อมูลเป็นมาตรฐาน (Normalize Data)
การปรับน้ำหนักให้เข้ากับข้อมูล (back-propagation ซึ่งจะครอบคลุมในบทเรียนสัปดาห์หน้า) จะดำเนินการได้เร็วขึ้นหากข้อมูลถูกทำให้เป็นมาตรฐาน นี่คือขั้นตอนเดียวกับที่คุณใช้ในหลักสูตร 1 ซึ่งคุณได้ทำให้คุณลักษณะในข้อมูลแต่ละรายการเป็นมาตรฐานเพื่อให้มีช่วงที่คล้ายกัน
ขั้นตอนต่อไปนี้ใช้ Keras [normalization layer](https://keras.io/api/layers/preprocessing_layers/numerical/normalization/). มีขั้นตอนดังต่อไปนี้:

- สร้าง "Normalization Layer" โปรดทราบว่าตามที่ใช้ในที่นี้ นี่ไม่ใช่ชั้นในโมเดลของคุณ
- 'ปรับ' ข้อมูล ขั้นตอนนี้จะเรียนรู้ค่าเฉลี่ยและความแปรปรวนของชุดข้อมูลและบันทึกค่าเหล่านั้นภายใน
- ทำการปรับขนาดข้อมูล (Normalize)


In [ ]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min post normalization: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

คัดลอกข้อมูลของเราเพื่อเพิ่มขนาดชุดฝึกอบรมและลดจำนวนรอบการฝึกอบรม

In [ ]:
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape)   

## Tensorflow Model

### Model
   <center> <img  src="./images/C2_W1_RoastingNetwork.PNG" width="200" />   <center/>  
มาสร้าง "เครือข่ายการคั่วกาแฟ" ตามที่อธิบายไว้ในบรรยายกันเถอะ เครือข่ายนี้มีสองเลเยอร์โดยใช้ฟังก์ชันการกระตุ้นซิกโมิด ดังภาพด้านล่าง:

In [ ]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

>**โน๊ต 1:** การกำหนดรูปข้อมูลอินพุตด้วย `tf.keras.Input(shape=(2,)),` บรรทัดนี้ระบุรูปข้อมูลที่คาดว่าจะได้รับเป็นอินพุต ซึ่งช่วยให้ TensorFlow สามารถกำหนดขนาดของพารามิเตอร์น้ำหนัก (weights) และ (bias) ได้ตั้งแต่เนิ่นต้น  ฟังก์ชันนี้มีประโยชน์สำหรับการศึกษาโมเดล TensorFlow ถึงแม้ว่าเราสามารถละเว้นการระบุรูปข้อมูลอินพุตในบรรทัดนี้ได้  TensorFlow จะทำการกำหนดขนาดของพารามิเตอร์เครือข่ายเองเมื่อระบุข้อมูลอินพุตในคำสั่ง  `model.fit` 

>**โน๊ต 2:** การใช้ฟังก์ชันกระตุ้น (activation function) sigmoid ในชั้นสุดท้ายไม่ถือเป็นแนวทางปฏิบัติที่ดีที่สุด
การใช้ฟังก์ชันกระตุ้น sigmoid ในชั้นสุดท้ายจะถูกแทนที่ด้วยการคำนวณฟังก์ชันสูญเสีย (loss function) แทน ซึ่งจะช่วยปรับปรุงความเสถียรทางตัวเลข  แนวคิดนี้จะได้รับการอธิบายเพิ่มเติมในห้องปฏิบัติการต่อไป

บรรทัด  `model.summary()` แสดงรายละเอียดของเครือข่าย:

In [ ]:
model.summary()

จำนวนพารามิเตอร์ที่แสดงในสรุปตรงกับจำนวนองค์ประกอบในอาร์เรย์น้ำหนักและอคติตามที่แสดงด้านล่าง

In [ ]:
L1_num_params = 2 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

มาตรวจสอบน้ำหนักและไบแอสที่ TensorFlow ได้สร้างขึ้น น้ำหนัก $W$ ควรมีขนาด (จำนวนคุณลักษณะในอินพุต, จำนวนหน่วยในเลเยอร์) ในขณะที่ไบแอส b ควรมีขนาดที่ตรงกับจำนวนหน่วยในเลเยอร์:
- ในเลเยอร์แรกที่มี 3 หน่วย เราคาดหวังว่า W จะมีขนาด (2,3) และ $b$ ควรมี 3 องค์ประกอบ
- ในเลเยอร์แรกที่มี 3 หน่วย เราคาดหวังว่า W จะมีขนาด (2,3) และ $b$ ควรมี 3 องค์ประกอบ

In [ ]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

ข้อความที่เหลือจะอธิบายเพิ่มเติมในสัปดาห์ที่ 2 สำหรับตอนนี้:
- คำสั่ง `model.compile` จะกำหนดฟังก์ชันการสูญเสีย (loss function) และระบุการปรับแต่งการรวบรวม (compile optimization)
- คำสั่ง `model.fit` จะรันการไล่ระดับ (gradient descent) และปรับน้ำหนักให้พอดีกับข้อมูล

In [ ]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

#### Epochs and batches
ในคำสั่ง`fit` ข้างต้น จำนวน `epochs` ถูกตั้งค่าเป็น 10 ซึ่งระบุว่าควรใช้ชุดข้อมูลทั้งหมดในการฝึก 10 ครั้ง ระหว่างการฝึก คุณจะเห็นผลลัพธ์ที่อธิบายความคืบหน้าของการฝึกดังนี้:
```
Epoch 1/10
6250/6250 [==============================] - 6s 910us/step - loss: 0.1782
```
ในบรรทัดเเรก `Epoch 1/10`,  หมายถึงโมเดลกำลังรันอยู่ในรอบที่ 1 จากทั้งหมด 10 รอบ. เพื่อประสิทธิภาพในการฝึกโมเดล ข้อมูลการฝึกจะถูกแบ่งออกเป็นกลุ่มย่อย (batches) โดยขนาดของกลุ่มย่อยมาตรฐานใน TensorFlow คือ 32. มีข้อมูลตัวอย่างทั้งหมด 200,000 ตัวอย่างในชุดข้อมูลที่ขยาย หรือเทียบเท่ากับ 6,250 กลุ่มย่อย. บรรทัดที่สอง `6250/6250 [====` กำลังแสดงว่ากลุ่มย่อยใดถูกประมวลผลแล้ว

#### น้ำหนักที่อัปเดตแล้ว
หลังจากการปรับค่า น้ำหนักได้ถูกอัปเดตแล้ว

In [ ]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

คุณจะเห็นว่าค่าต่างๆ นั้นแตกต่างไปจากที่คุณพิมพ์ไว้ก่อนเรียกใช้  `model.fit()`. ด้วยค่าเหล่านี้ โมเดลน่าจะสามารถแยกแยะได้ว่ากาแฟคั่วแบบไหนดีหรือไม่ดี

สำหรับการสนทนาครั้งต่อไป แทนที่จะใช้น้ำหนักที่คุณเพิ่งได้รับ คุณจะตั้งน้ำหนักบางอย่างที่เราบันทึกไว้จากการฝึกครั้งก่อนก่อน เพื่อให้สมุดบันทึกนี้ยังคงแข็งแกร่งต่อการเปลี่ยนแปลงของ Tensorflow ไปตามกาลเวลา การฝึกอบรมที่แตกต่างกันอาจให้ผลลัพธ์ที่แตกต่างกันเล็กน้อย บทสนทนาต่อไปนี้ใช้ได้เมื่อโมเดลมีน้ำหนักที่คุณจะโหลดด้านล่าง

อย่าลังเลที่จะรันสมุดบันทึกอีกครั้งในภายหลังโดยปิดใช้งานเซลล์ด้านล่างเพื่อดูว่ามีข้อแตกต่างหรือไม่ หากคุณได้รับค่าความสูญเสียที่ต่ำหลังการฝึกอบรมข้างต้น (เช่น 0.002) คุณน่าจะได้รับผลลัพธ์เดียวกัน

In [ ]:
# After finishing the lab later, you can re-run all 
# cells except this one to see if your trained model
# gets the same results.

# Set weights from a previous run. 
W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])

# Replace the weights from your trained model with
# the values above.
model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])

In [ ]:
# Check if the weights are successfully replaced
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

### Predictions
<img align="left" src="./images/C2_W1_RoastingDecision_v1.png"     style=" width:380px; padding: 10px 20px; " >

เมื่อคุณมีโมเดลที่ได้รับการฝึกฝนแล้ว คุณสามารถใช้โมเดลนั้นเพื่อทำการคาดการณ์ได้ จำไว้ว่าผลลัพธ์ของโมเดลของเราคือความน่าจะเป็น ในกรณีนี้ ความน่าจะเป็นของการคั่วกาแฟที่ดี เพื่อทำการตัดสินใจ คุณต้องนำความน่าจะเป็นไปใช้กับเกณฑ์ ในกรณีนี้ เราจะใช้ 0.5

เรามาเริ่มสร้างข้อมูลอินพุตกันก่อน โมเดลคาดหวังตัวอย่างหนึ่งตัวอย่างหรือมากกว่านั้น โดยตัวอย่างจะอยู่ในแถวของเมทริกซ์ ในกรณีนี้ เรามีสองฟีเจอร์ ดังนั้นเมทริกซ์จะเป็น (m,2) โดยที่ m คือจำนวนตัวอย่าง
เพื่อทำการทำนาย คุณจะใช้เมธอด `predict` 

In [ ]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

เพื่อแปลงความน่าจะเป็นให้เป็นการตัดสินใจ เราใช้เกณฑ์ (threshold) ดังนี้:

In [ ]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

สามารถทำได้อย่างรวบรัดกว่านี้:

In [ ]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

## ฟังก์ชันเลเยอร์ (Layer Functions)

มาตรวจสอบฟังก์ชันของหน่วยเพื่อกำหนดบทบาทของหน่วยเหล่านั้นในการตัดสินใจการคั่วกาแฟ เราจะพล็อตผลลัพธ์ของแต่ละโหนดสำหรับค่าทั้งหมดของอินพุต (ระยะเวลา อุณหภูมิ) แต่ละหน่วยเป็นฟังก์ชันโลจิสติกที่มีผลลัพธ์อยู่ในช่วงศูนย์ถึงหนึ่ง การแรเงาในกราฟแสดงถึงค่าผลลัพธ์
> หมายเหตุ: ในห้องปฏิบัติการ เรามักจะเริ่มนับสิ่งต่าง ๆ ที่ศูนย์ในขณะที่ในบรรยายอาจเริ่มจาก 1

In [ ]:
plt_layer(X,Y.reshape(-1,),W1,b1,norm_l)
plt.show()

"การแรเงาแสดงให้เห็นว่าแต่ละหน่วยรับผิดชอบ "พื้นที่การคั่วที่ไม่ดี" ที่แตกต่างกัน หน่วย 0 มีค่าที่สูงขึ้นเมื่ออุณหภูมิต่ำเกินไป หน่วย 1 มีค่าที่สูงขึ้นเมื่อระยะเวลาสั้นเกินไป และหน่วย 2 มีค่าที่สูงขึ้นสำหรับการรวมเวลา/อุณหภูมิที่ไม่ดี เป็นที่น่าสังเกตว่าเครือข่ายได้เรียนรู้ฟังก์ชันเหล่านี้ด้วยตนเองผ่านกระบวนการไล่ระดับ (gradient descent) ฟังก์ชันเหล่านี้มีความคล้ายคลึงกับฟังก์ชันที่บุคคลอาจเลือกใช้เพื่อตัดสินใจในลักษณะเดียวกัน

"การพล็อตฟังก์ชันของเลเยอร์สุดท้ายนั้นค่อนข้างยากที่จะมองเห็นได้ชัดเจน อินพุตของเลเยอร์สุดท้ายนี้คือเอาต์พุตของเลเยอร์แรก เราทราบว่าเลเยอร์แรกใช้ฟังก์ชันซิกมอยด์ (sigmoids) ดังนั้นเอาต์พุตของเลเยอร์แรกจึงอยู่ในช่วงระหว่างศูนย์ถึงหนึ่ง เราสามารถสร้างพล็อต 3 มิติที่คำนวณเอาต์พุตสำหรับทุก ๆ คอมบิเนชันที่เป็นไปได้ของอินพุตทั้งสามนี้ ดังแสดงด้านล่าง ด้านบน ค่าเอาต์พุตที่สูงสอดคล้องกับพื้นที่ 'การคั่วที่ไม่ดี' ด้านล่าง ค่าเอาต์พุตสูงสุดอยู่ในพื้นที่ที่อินพุตทั้งสามมีค่าต่ำสอดคล้องกับพื้นที่ 'การคั่วที่ดี'

In [ ]:
plt_output_unit(W2,b2)
plt.show()

กราฟสุดท้ายแสดงเครือข่ายทั้งหมดทำงาน
กราฟด้านซ้ายคือผลลัพธ์ดิบของเลเยอร์สุดท้ายที่แสดงโดยการแรเงดสีฟ้า ซึ่งทับซ้อนกับข้อมูลการฝึกที่แสดงโดย X และ O
กราฟด้านขวาคือผลลัพธ์ของเครือข่ายหลังจากผ่านเกณฑ์การตัดสินใจ X และ O ที่นี่สอดคล้องกับการตัดสินใจที่ทำโดยเครือข่าย
ขั้นตอนต่อไปนี้ใช้เวลาสักครู่ในการทำงาน

In [ ]:
netf= lambda x : model.predict(norm_l(x))
plt_network(X,Y,netf)
plt.show()

## ยินดีด้วย!

คุณได้สร้างเครือข่ายประสาทเทียมขนาดเล็กใน TensorFlow แล้ว

เครือข่ายนี้แสดงให้เห็นถึงความสามารถของเครือข่ายประสาทเทียมในการจัดการการตัดสินใจที่ซับซ้อน โดยการแบ่งการตัดสินใจระหว่างหน่วยหลายหน่วย